In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


import patsy as pat
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests


In [49]:
def standardize(mask,data):
    scaler = StandardScaler(with_mean=False, with_std=True)
    scaler.fit(data[mask])
    standardized=scaler.transform(data)
    return standardized

def case_control(pheno,case,regressors,conn,std=False):
    """
    pheno = dataframe:
        -filtered to be only relevant subjects for case control (use mask_cc)
        -case column is onehot encoded
    case = column from pheno
    regressors = list of strings, formatted for patsy
    connectomes = n_subjects x n_edges array
    
    Returns:
    table = n_edges
        - betas = the difference between case + control
        - betas_std = including standardization on controls
        - pvalues = pvalues
        - qvalues = fdr corrected pvalues alpha = 0.05
    """
    n_edges = conn.shape[1]

    betas = np.zeros(n_edges)
    betas_std = np.zeros(n_edges)
    pvalues = np.zeros(n_edges)

    formula = ' + '.join((regressors + [case]))
    dmat = pat.dmatrix(formula, pheno, return_type='dataframe',NA_action='raise')
    
    mask_std = ~pheno[case].to_numpy(dtype=bool)
    conn_std = standardize(mask_std, conn)
    
    for edge in range(n_edges): 
        model = sm.OLS(conn.iloc[:,edge],dmat)
        results = model.fit()
        betas[edge] = results.params[case]
        pvalues[edge] = results.pvalues[case]

        if std:
            model_std = sm.OLS(conn_std[:,edge],dmat)
            results_std = model_std.fit()
            betas_std[edge] = results_std.params[case]
        
    mt = multipletests(pvalues,method='fdr_bh')
    reject = mt[0]
    qvalues = mt[1]
    
    table = pd.DataFrame(np.array([betas,betas_std,pvalues,qvalues,reject]).transpose(),
                         columns=['betas','betas_std','pvalues','qvalues','reject'])
    return table

In [33]:
p_pheno = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/pheno_01-12-21.csv'
p_conn = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/connectomes_01-12-21.csv'

pheno = pd.read_csv(p_pheno, index_col=0)
conn = pd.read_csv(p_conn,index_col=0)

/home/harveyaa/miniconda3/envs/MTL/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
cases = ['SZ',
        'ASD',
        'BIP',
        'DEL22q11_2',
        'DUP22q11_2',
        'DEL16p11_2',
        'DUP16p11_2',
        'DEL1q21_1',
        'DUP1q21_1']

conf = ['AGE',
        'SEX',
        'SITE',
        'mean_conn',
        'FD_scrubbed']

In [51]:
p_ids = '/home/harveyaa/Documents/masters/neuropsych_mtl/datasets/cv_folds/hybrid'

ids = {}
for case in cases:
    ids[case] = pd.read_csv(os.path.join(p_ids,f"{case}.csv"),index_col=0)

In [72]:
mtd_folds_train = {}
for case in cases:
    df = ids[case]
    mtds = []
    print(case)
    for i in range(5):
        mask = pheno.index.isin(df[df[f'fold_{i}']==0].index)
        #cc = case_control(pheno[mask],case,conf,conn[mask],std=False)
        cc = case_control(pheno[mask],case,conf,conn[mask],std=True)

        rank = pd.qcut(np.abs(cc['betas_std']),10,labels=False)                    
        decile = np.abs(cc['betas_std'])[rank[rank==9]]
        mtd = np.mean(decile)
        mtds.append(mtd)

        print(f'Fold {i}: ',np.round(mtd,2))
        #sns.distplot(cc['betas_std'])
        #plt.title(case)
        #plt.show()
    mtd_folds_train[case] = mtds

SZ
Fold 0:  0.07
Fold 1:  0.08
Fold 2:  0.09
Fold 3:  0.11
Fold 4:  0.11
ASD
Fold 0:  0.1
Fold 1:  0.08
Fold 2:  0.1
Fold 3:  0.09
Fold 4:  0.06
BIP
Fold 0:  0.31
Fold 1:  0.34
Fold 2:  0.32
Fold 3:  0.38
Fold 4:  0.35
DEL22q11_2
Fold 0:  0.34
Fold 1:  0.47
Fold 2:  0.29
Fold 3:  0.7
Fold 4:  0.21
DUP22q11_2
Fold 0:  0.21
Fold 1:  0.22
Fold 2:  0.27
Fold 3:  0.26
Fold 4:  0.32
DEL16p11_2
Fold 0:  0.11
Fold 1:  0.01
Fold 2:  0.27
Fold 3:  0.02
Fold 4:  0.14
DUP16p11_2
Fold 0:  0.24
Fold 1:  0.22
Fold 2:  0.21
Fold 3:  0.26
Fold 4:  0.27
DEL1q21_1
Fold 0:  0.03
Fold 1:  0.02
Fold 2:  0.08
Fold 3:  0.06
Fold 4:  0.08
DUP1q21_1
Fold 0:  0.04
Fold 1:  0.08
Fold 2:  0.09
Fold 3:  0.07
Fold 4:  0.12


In [78]:
df_mtd_folds_train = 10*pd.DataFrame(mtd_folds_train)
df_mtd_folds_train

,SZ,ASD,BIP,DEL22q11_2,DUP22q11_2,DEL16p11_2,DUP16p11_2,DEL1q21_1,DUP1q21_1
0,0.680994,0.964244,3.076685,3.431407,2.090300,1.112586,2.378542,0.290457,0.437681
1,0.814477,0.811996,3.372839,4.664975,2.198352,0.069134,2.239264,0.200002,0.820121
2,0.931953,0.987900,3.244960,2.865920,2.674040,2.728816,2.096516,0.763954,0.942408
3,1.104631,0.899731,3.822516,7.010751,2.643065,0.154874,2.619748,0.565534,0.727782
4,1.149830,0.573223,3.452204,2.135809,3.239847,1.428678,2.716533,0.795664,1.186412


In [74]:
mtd_folds_test = {}
for case in cases:
    df = ids[case]
    mtds = []
    print(case)
    for i in range(5):
        mask = pheno.index.isin(df[df[f'fold_{i}']==1].index)
        #cc = case_control(pheno[mask],case,conf,conn[mask],std=False)
        cc = case_control(pheno[mask],case,conf,conn[mask],std=True)

        rank = pd.qcut(np.abs(cc['betas_std']),10,labels=False)                    
        decile = np.abs(cc['betas_std'])[rank[rank==9]]
        mtd = np.mean(decile)
        mtds.append(mtd)

        print(f'Fold {i}: ',np.round(mtd,2))
        #sns.distplot(cc['betas_std'])
        #plt.title(case)
        #plt.show()
    mtd_folds_test[case] = mtds

SZ
Fold 0:  0.05
Fold 1:  0.16
Fold 2:  0.04
Fold 3:  0.03
Fold 4:  0.02
ASD
Fold 0:  0.03
Fold 1:  0.09
Fold 2:  0.0
Fold 3:  0.03
Fold 4:  0.2
BIP
Fold 0:  0.7
Fold 1:  0.2
Fold 2:  0.43
Fold 3:  0.03
Fold 4:  0.0
DEL22q11_2
Fold 0:  0.35
Fold 1:  0.35
Fold 2:  0.46
Fold 3:  0.88
Fold 4:  0.06
DUP22q11_2
Fold 0:  1.93
Fold 1:  0.5
Fold 2:  1.9
Fold 3:  1.27
Fold 4:  1.16
DEL16p11_2
Fold 0:  0.19
Fold 1:  1.56
Fold 2:  1.04
Fold 3:  0.27
Fold 4:  0.35
DUP16p11_2
Fold 0:  0.61
Fold 1:  0.27
Fold 2:  0.44
Fold 3:  0.29
Fold 4:  0.08
DEL1q21_1
Fold 0:  0.29
Fold 1:  0.03
Fold 2:  0.57
Fold 3:  0.15
Fold 4:  0.04
DUP1q21_1
Fold 0:  0.12
Fold 1:  0.25
Fold 2:  0.3
Fold 3:  0.12
Fold 4:  0.34


In [79]:
df_mtd_folds_test = 10*pd.DataFrame(mtd_folds_test)
df_mtd_folds_test

,SZ,ASD,BIP,DEL22q11_2,DUP22q11_2,DEL16p11_2,DUP16p11_2,DEL1q21_1,DUP1q21_1
0,0.527303,0.332918,6.959817,3.531932,19.301735,1.905041,6.146923,2.869940,1.219897
1,1.577464,0.907880,1.996789,3.457533,4.977620,15.552953,2.658040,0.299526,2.468792
2,0.415529,0.001303,4.346699,4.645787,19.022878,10.357076,4.375184,5.695802,3.007191
3,0.290203,0.318188,0.333508,8.805149,12.715295,2.685385,2.857059,1.523922,1.231525
4,0.163639,1.975101,0.018155,0.635421,11.550748,3.475453,0.761267,0.351478,3.370880


In [80]:
df_mtd_folds_train - df_mtd_folds_test

,SZ,ASD,BIP,DEL22q11_2,DUP22q11_2,DEL16p11_2,DUP16p11_2,DEL1q21_1,DUP1q21_1
0,0.153691,0.631326,-3.883132,-0.100525,-17.211436,-0.792455,-3.768381,-2.579482,-0.782215
1,-0.762987,-0.095885,1.376050,1.207442,-2.779268,-15.483819,-0.418775,-0.099523,-1.648671
2,0.516424,0.986597,-1.101739,-1.779867,-16.348838,-7.628260,-2.278668,-4.931848,-2.064783
3,0.814428,0.581542,3.489009,-1.794398,-10.072230,-2.530510,-0.237311,-0.958389,-0.503743
4,0.986191,-1.401879,3.434049,1.500388,-8.310901,-2.046776,1.955265,0.444187,-2.184468


# Summary
Cant exclude any folds for an obvious reason
All have some signal in case control
Not total agreement b/w classifiers that can't predict